## Basics

In [1]:
import torch

from torchonometrics.linear import LinearRegression
from torchonometrics.mle import LogisticRegression, PoissonRegression

In [2]:
cuda_available = torch.cuda.is_available()
if cuda_available:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

device

device(type='cuda')

## OLS

In [3]:
torch.manual_seed(42)
n, p = 100, 5
X = torch.randn(n, p)
true_coef = torch.randn(p)
y = X @ true_coef + 0.1 * torch.randn(n)

model = LinearRegression()
model.fit(X, y, se="HC1")

model.summary()

LinearRegression Results
coef: tensor([-0.1935,  0.0301, -1.3814, -0.3106,  0.9807], device='cuda:0')
se: tensor([0.0097, 0.0082, 0.0124, 0.0111, 0.0123], device='cuda:0')


In [4]:
# Panel data setup
n_firms, n_years = 100, 10
n_obs = n_firms * n_years

# Generate data with firm and year effects
X = torch.randn(n_obs, 3)
firm_ids = torch.repeat_interleave(torch.arange(n_firms), n_years)
year_ids = torch.tile(torch.arange(n_years), (n_firms,))

# Add intercept

# True coefficients and effects
true_coef = torch.tensor([1.5, -0.8, 0.3])
firm_effects = torch.randn(n_firms)[firm_ids]
year_effects = torch.randn(n_years)[year_ids]

y = X @ true_coef + firm_effects + year_effects + 0.1 * torch.randn(n_obs)

# Fit with two-way fixed effects
model = LinearRegression()
model.fit(X, y, fe=[firm_ids, year_ids])
print(f"Coefficients: {model.params['coef']}")

Coefficients: tensor([ 1.4982, -0.7990,  0.2962], device='cuda:0', dtype=torch.float64)


## logit

In [5]:
# Binary classification data
n, p = 10_000_000, 4
X = torch.randn(n, p)
X_with_intercept = torch.cat([torch.ones(n, 1), X], dim=1)

# Generate binary outcomes
true_coef = torch.tensor([0.5, 1.0, -0.8, 0.3, 0.2])
logits = X_with_intercept @ true_coef
probs = torch.sigmoid(logits)
y = torch.bernoulli(probs)

# Fit logistic regression
model = LogisticRegression(maxiter=100)


In [6]:
%%time
model.fit(X_with_intercept, y)
model.summary()

LogisticRegression Results
Optimizer: LBFGS
Optimization: 12/100 iterations
Final Log-Likelihood: -5383595.0000
No. Observations: 10000000

Variable     Coef.      Std.Err.   t        P>|t|    [95.0%   Conf. Interval]
----------------------------------------------------------------------
const        0.4996     0.0008     658.489  0.000    0.498    0.501   
x1           0.9987     0.0009     1134.631 0.000    0.997    1.000   
x2           -0.7991    0.0008     -958.960 0.000    -0.801   -0.797  
x3           0.3006     0.0008     397.342  0.000    0.299    0.302   
x4           0.1999     0.0007     266.781  0.000    0.198    0.201   
CPU times: user 1.65 s, sys: 135 ms, total: 1.79 s
Wall time: 1.79 s


In [8]:
model.device

device(type='cuda')

In [9]:
# Predictions
y_pred_proba = model.predict_proba(X_with_intercept)
y_pred = model.predict(X_with_intercept)
torch.unique(y_pred, return_counts=True)

(tensor([0, 1], device='cuda:0', dtype=torch.int32),
 tensor([3533838, 6466162], device='cuda:0'))

## poisson

In [10]:
torch.manual_seed(42)
n, p = 10_000_000, 2
X = torch.randn(n, p)
X_with_intercept = torch.cat([torch.ones(n, 1), X], dim=1)
true_coef = torch.tensor([1.0, 0.5, -0.3])

# Generate Poisson counts
linear_pred = X_with_intercept @ true_coef
lambda_true = torch.exp(linear_pred)
y = torch.poisson(lambda_true)

In [11]:
%%time
model = PoissonRegression(maxiter=50)
model.fit(X_with_intercept, y)
model.summary()

PoissonRegression Results
Optimizer: LBFGS
Optimization: 12/50 iterations
Final Log-Likelihood: 10960920.0000
No. Observations: 10000000

Variable     Coef.      Std.Err.   t        P>|t|    [95.0%   Conf. Interval]
----------------------------------------------------------------------
const        1.0000     0.0002     4903.634 0.000    1.000    1.000   
x1           0.5000     0.0002     2837.848 0.000    0.500    0.500   
x2           -0.3000    0.0002     -1702.948 0.000    -0.300   -0.300  
CPU times: user 917 ms, sys: 2.67 ms, total: 920 ms
Wall time: 921 ms
